In [55]:
import pandas as pd
import numpy as np
import logging
import pickle
from datetime import datetime,timedelta
import urllib
from io import StringIO
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models.layouts import Column
from bokeh.models.sources import ColumnDataSource
from bokeh.models import Span, Label, DatetimeTickFormatter
from bokeh.models.tools import HoverTool,BoxZoomTool,CrosshairTool,ResetTool,PanTool,WheelZoomTool
from ipywidgets import IntSlider, DatePicker, interact, interact_manual, interactive, fixed
from IPython.display import display

output_notebook()

logging.basicConfig(format='%(levelname)s:\t%(message)s', level=logging.INFO)

# import ipywidgets as widgets
# logging.info(sorted(widgets.Widget.widget_types))

# import ipywidgets
# logging.info(ipywidgets.__version__)

Loading BokehJS ...

In [56]:
load_flow_data_pkl = False
load_flow_data_pkl = True

if load_flow_data_pkl:
    with open("us.pkl",'rb') as fin:
        upstream_df = pickle.load(fin)
    with open("ds.pkl",'rb') as fin:
        downstream_df = pickle.load(fin)
else:
    upstream_df = pd.read_excel("M-10 Meter 00-35-1 Data.xlsx",sheetname="M-10")
    downstream_df = pd.read_excel("3.b. CSO021Overflow_FlowData_Dec2014-Oct2015.xlsx",sheetname="CSO 021 Overflow")

    upstream_df['Time'] = pd.to_datetime(upstream_df['Time'],format="%Y-%m-%d %H-%M-%S")
    upstream_df.set_index("Time",inplace=True)

    downstream_df['Time'] = pd.to_datetime(downstream_df['Time'],format="%Y-%m-%d %H-%M-%S")
    downstream_df.set_index("Time",inplace=True)

    with open("us.pkl",'wb') as fout:
        pickle.dump(upstream_df,fout)
    with open("ds.pkl",'wb') as fout:
        pickle.dump(downstream_df,fout)
        
def plot_level_flow_alarm(upstream_df,downstream_df,proposed_alarm_elevation,start_date,end_date):
    
    if end_date < start_date:
        temp_date = start_date
        start_date = end_date
        end_date = temp_date
    
    upstream_df = upstream_df.reset_index()
    downstream_df = downstream_df.reset_index()

    upstream_df = upstream_df[upstream_df['Time'] <= end_date]
    downstream_df = downstream_df[downstream_df['Time'] <= end_date]
    upstream_df = upstream_df[upstream_df['Time'] >= start_date]
    downstream_df = downstream_df[downstream_df['Time'] >= start_date]
    
    level_hover_tool = HoverTool(
            tooltips=[
                ("Elevation (in)", "@{Elevation (in)}"),
                ("Time", "@{String Time}"),
            ],
            line_policy='nearest',
            names=['data']
        )

    flow_hover_tool = HoverTool(
            tooltips=[
                ("Flow (cf/s)", "@{Flow (cf/s)}"),
                ("Time", "@{String Time}"),
            ],
            line_policy='nearest',
            names=['data']
        )

    alarm_hover_tool = HoverTool(
            tooltips=[
                ("Elevation (in)", "@elevations"),
                ("Number of Alarms", "@alarms"),
            ],
            line_policy='nearest',
        )

    width = 850
    level_figure = figure(width=width, height=250, x_axis_type="datetime",tools=[level_hover_tool,BoxZoomTool(),CrosshairTool(),ResetTool(),PanTool(),WheelZoomTool()])
    flow_figure = figure(width=width, height=250, x_axis_type="datetime",x_range=level_figure.x_range,tools=[flow_hover_tool,BoxZoomTool(),CrosshairTool(),ResetTool(),PanTool(),WheelZoomTool()])
    alarm_figure = figure(width=width, height=250,tools=[alarm_hover_tool,BoxZoomTool(),CrosshairTool(),ResetTool(),PanTool(),WheelZoomTool()],y_axis_type='log',y_range=[1,5000])

    upstream_df['Elevation (in)'] = upstream_df['Level (in)'] + -0.6*12
    upstream_df['String Time'] = upstream_df['Time'].dt.strftime('%HH:%MM')
    downstream_df['String Time'] = downstream_df['Time'].dt.strftime('%HH:%MM')

    us_cds = ColumnDataSource(upstream_df)
    ds_cds = ColumnDataSource(downstream_df)

    default_time_formatter = DatetimeTickFormatter()
    default_time_formatter.minutes = ["%HH:%MM %m/%d"]
    default_time_formatter.hourmin = ["%HH:%MM %m/%d"]
    default_time_formatter.hours = ["%HH:%MM %m/%d"]
    default_time_formatter.days = ["%d %b %Y"]
    default_time_formatter.months = ["%d %b %Y"]
    default_time_formatter.years = ["%d %b %Y"]

    level_figure.xaxis.formatter = default_time_formatter
    flow_figure.xaxis.formatter = default_time_formatter

    level_figure.line(x='Time',y='Elevation (in)',source=us_cds,color='#66c2a5')
    level_figure.circle(x='Time',y='Elevation (in)',source=us_cds,color='#66c2a5',name='data')

    overflow_events = downstream_df[downstream_df['Flow (cf/s)']>0].index
    overflow_event_diffs = np.diff(overflow_events.values)
    overflow_start_arg = np.hstack((0,np.argwhere(overflow_event_diffs>1).reshape(-1)+1))
    overflow_end_arg = np.hstack((np.argwhere(overflow_event_diffs>1).reshape(-1),len(overflow_events)-1))

    for (start,end) in zip(overflow_events[overflow_start_arg],overflow_events[overflow_end_arg]):
        start -= 1
        end += 0
        temp_cds = ColumnDataSource(downstream_df.loc[start:end+1])
        flow_figure.line(x='Time',y='Flow (cf/s)',source=temp_cds,color='#fc8d62')
        flow_figure.circle(x='Time',y='Flow (cf/s)',source=temp_cds,color='#fc8d62',name='data')

    levels = []
    alarms = []
    # print(upstream_df['Level (in)'].max())
    for level in np.arange(0,upstream_df['Level (in)'].max(),1):
        alarm_events = upstream_df[upstream_df['Level (in)']>=level].index
        alarm_event_diffs = np.diff(alarm_events.values)
        alarm_start_arg = np.hstack((0,np.argwhere(alarm_event_diffs>1).reshape(-1)+1))
        alarm_end_arg = np.hstack((np.argwhere(alarm_event_diffs>1).reshape(-1),len(alarm_events)-1))


        if len(alarm_start_arg) > 0:
            levels.append(level)
            alarms.append(len(alarm_start_arg))

    levels = np.array(levels)
    alarms = np.array(alarms)

    elevations = levels - 0.6*12.

    alarm_cds = ColumnDataSource({'elevations':elevations.astype(float),'alarms':alarms.astype(float)})
    alarm_figure.line(x='elevations',y='alarms',color='#8da0cb',source=alarm_cds)
    alarm_figure.circle(x='elevations',y='alarms',color='#8da0cb',source=alarm_cds)

    actual_overflows = Span(location=len(overflow_start_arg),
                                dimension='width', line_color='#e41a1c', line_width=3)
    alarm_figure.add_layout(actual_overflows)
    alarm_label = Label(x=0,y=len(overflow_start_arg),text="Actual Number of Overflows",y_offset=3,text_font_size='8pt')
    alarm_figure.add_layout(alarm_label)

    current_float_elevation = 12*(14/12+3.58+0.6)
    current_float = Span(location=current_float_elevation,dimension='height', line_color='#ff7f00', line_width=3)
    alarm_figure.add_layout(current_float)
    float_label = Label(x=current_float_elevation,y=0,text="Current Float Switch",x_offset=-3,angle=np.pi/2,text_font_size='8pt')
    alarm_figure.add_layout(float_label)
    
    proposed_alarm = Span(location=proposed_alarm_elevation,dimension='height', line_color='#984ea3', line_width=3, line_dash='dashed')
    alarm_figure.add_layout(proposed_alarm)
    alarm_label = Label(x=proposed_alarm_elevation,y=0,text="Proposed Alarm Elevation",x_offset=-3,angle=np.pi/2,text_font_size='8pt')
    alarm_figure.add_layout(alarm_label)
    
    for level in [proposed_alarm_elevation]:
        alarm_events = upstream_df[upstream_df['Elevation (in)']>=level].index
        alarm_event_diffs = np.diff(alarm_events.values)
        alarm_start_arg = np.hstack((0,np.argwhere(alarm_event_diffs>1).reshape(-1)+1))
        alarm_end_arg = np.hstack((np.argwhere(alarm_event_diffs>1).reshape(-1),len(alarm_events)-1))

        for (start,end) in zip(alarm_events[alarm_start_arg],alarm_events[alarm_end_arg]):
            level_figure.line(x=upstream_df.loc[start:end+1,'Time'],y=upstream_df.loc[start:end+1,'Elevation (in)'],color='#e34a33',line_width=3)

        float_label = Label(x=100, y=200, text="Red Highlights Show Alarms at Proposed Alarm Elevation", text_font_size='8pt', x_units='screen', y_units='screen')
        level_figure.add_layout(float_label)
        current_float = Span(location=level,dimension='width', line_color='#e34a33', line_width=1, line_dash='dashed')
        level_figure.add_layout(current_float)

    level_figure.title.text = "2015 Historical Upstream Level"
    flow_figure.title.text = "2015 Historical Downstream Flow"
    alarm_figure.title.text = "Number of Alarms vs. Alarm Level"

    level_figure.yaxis.axis_label = 'Upstream Elevation (in)'
    level_figure.xaxis.axis_label = 'Date'
    flow_figure.yaxis.axis_label = 'Downstream Flow (cf/s)'
    flow_figure.xaxis.axis_label = 'Date'
    alarm_figure.yaxis.axis_label = 'Number of Alarms'
    alarm_figure.xaxis.axis_label = 'Alarm Set Level Elevation (in)'

    # Column(level_figure,flow_figure,alarm_figure)
    show(Column(level_figure,flow_figure,alarm_figure))
    
    return

In [58]:
interact_manual(plot_level_flow_alarm, upstream_df=fixed(upstream_df), downstream_df=fixed(downstream_df), proposed_alarm_elevation=IntSlider(value=61,description="Pick an Alarm Elevation"), start_date=DatePicker(description="Pick a Start Date",value=upstream_df.index.min()),end_date=DatePicker(description="Pick a End Date",value=upstream_df.index.max()))

<function __main__.plot_level_flow_alarm>